In [1]:
import sys
import os
import yaml
import argparse
import logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s:%(message)s')

import onnx
import onnxruntime

from lightning import Trainer
from pytorch_lightning.loggers import CSVLogger
import torch
from torch_geometric.loader import DataLoader

from Networks.Embedding.embedding import Embedding
from Networks.GNN.InteractionNetwork import InteractionNetwork
from Networks.utils import *


INFO:Loading faiss with AVX512 support.
INFO:Successfully loaded faiss with AVX512 support.


### Export Embedding model

In [2]:
d = torch.load("Datasets/stop_M600_588_ct200_2018.pt")
#model = Embedding.load_from_checkpoint("EMB_out_0502_2/model-epoch=57-val_loss=0.00285.ckpt")
model = Embedding.load_from_checkpoint("EMB_0603_3/model-epoch=105-val_loss=0.00292.ckpt")
input_sample = d[0].x.float()
model.to_onnx('EMB_0603_3.onnx', input_sample, input_names=['input_tk'], output_names=['emb'], export_params=True,opset_version=12,dynamic_axes={"input_tk":[0]})

/users/ang.li/anaconda3/envs/gnn_cpu/lib/python3.9/site-packages/torch/onnx/utils.py:2098: UserWarning: No names were found for specified dynamic axes of provided input.Automatically generated names will be applied to each dynamic axes of input input_tk
  warnings.warn(
/users/ang.li/public/SoftDV/ML/GNN_SDV_Reco/Networks/Embedding/embedding.py:76: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  scale_mean = torch.tensor([1.6466, -0.0012, -0.0039, -0.1067, 0.0449, -0.0283, 0.1674, 12.7531, 0.0178, 1.1603, 14.7395])
/users/ang.li/public/SoftDV/ML/GNN_SDV_Reco/Networks/Embedding/embedding.py:77: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to c

### Export GNN model

In [100]:
dgnn = torch.load("EMB_out_0502_2_output_rtest0p15_filtered_newweight/train/stop_M600_588_ct200_2018.pt")
input_sample = tuple((dgnn[0].x.float(),dgnn[0].pred_edges_emb))
model = InteractionNetwork.load_from_checkpoint("../GNN_SDV_Reco_localtest/GNN_dist0p15_0509/model-epoch=198-val_loss=0.45724.ckpt")
model.to_onnx('GNN_1.onnx', input_sample, input_names=['input_tk','input_edges'], output_names=['gnn'], export_params=True,opset_version=12,dynamic_axes={"input_tk":[0],"input_edges":[1]})

In [101]:
ort_session = onnxruntime.InferenceSession('GNN_1.onnx')

2024-05-22 16:14:23.425573183 [E:onnxruntime:Default, env.cc:254 ThreadMain] pthread_setaffinity_np failed for thread: 44229, index: 0, mask: {2, 3, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2024-05-22 16:14:23.425584819 [E:onnxruntime:Default, env.cc:254 ThreadMain] pthread_setaffinity_np failed for thread: 44230, index: 1, mask: {4, 5, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2024-05-22 16:14:23.425638652 [E:onnxruntime:Default, env.cc:254 ThreadMain] pthread_setaffinity_np failed for thread: 44233, index: 4, mask: {10, 11, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2024-05-22 16:14:23.425646261 [E:onnxruntime:Default, env.cc:254 ThreadMain] pthread_setaffinity_np failed for thread: 44234, index: 5, mask: {12, 13, }, error code: 22 error msg: Invalid argument. Speci

In [102]:
ort_session.get_inputs()[1].shape

[2, 'input_edges_dynamic_axes_1']

In [122]:
dgnn[5].pred_edges_emb.max()

tensor(47)

In [126]:
d[5].x.shape

torch.Size([44, 11])

In [129]:
ort_inputs = {'input_tk': np.array(dgnn[56].x.float()), 'input_edges': np.array(dgnn[56].pred_edges_emb)}
ort_outs = ort_session.run(None, ort_inputs)

In [136]:
ort_outs[0]
1/(1 + np.exp(-ort_outs[0])) 

array([0.00045452, 0.00036537, 0.00034495, ..., 0.00037103, 0.00041873,
       0.00034859], dtype=float32)

In [91]:
dgnn[0]

Data(x=[23, 11], true_edges=[2, 14], pred_edges_emb=[2, 302], distances_emb=[302], edges_y=[302], edge_weight=[302])

### Load Embedding model

In [56]:
ort_session = onnxruntime.InferenceSession('EMB_1.onnx')

2024-05-22 14:46:00.490341717 [E:onnxruntime:Default, env.cc:254 ThreadMain] pthread_setaffinity_np failed for thread: 18056, index: 0, mask: {2, 3, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2024-05-22 14:46:00.490394442 [E:onnxruntime:Default, env.cc:254 ThreadMain] pthread_setaffinity_np failed for thread: 18057, index: 1, mask: {4, 5, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2024-05-22 14:46:00.490533471 [E:onnxruntime:Default, env.cc:254 ThreadMain] pthread_setaffinity_np failed for thread: 18059, index: 3, mask: {8, 9, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2024-05-22 14:46:00.490664560 [E:onnxruntime:Default, env.cc:254 ThreadMain] pthread_setaffinity_np failed for thread: 18060, index: 4, mask: {10, 11, }, error code: 22 error msg: Invalid argument. Specify

In [60]:
input_name = ort_session.get_inputs()[0].name

In [67]:
np.array(d[15].x.float()).shape

(51, 11)

In [83]:
testinput = np.array(torch.tensor([[0,1,2,3,4,5,6,7,8,9,10]]).float())

In [64]:
ort_inputs = {input_name: np.array(d[15].x.float())}

In [84]:
ort_inputs = {input_name: testinput}

In [85]:
ort_outs = ort_session.run(None, ort_inputs)

In [66]:
ort_outs[0].shape

(51, 16)

In [14]:
type(ort_outs[0].shape[0])

int

In [86]:
ort_outs[0]

array([[ 0.40249282,  1.0423677 , -0.06209265, -0.1716794 , -0.23400308,
        -1.0785693 ,  1.0211241 , -0.08025236, -0.5017265 ,  0.07464715,
        -1.0424998 , -0.01701241,  0.87342536,  0.7729091 , -1.3853618 ,
        -0.30868497]], dtype=float32)

In [ ]:
input_name

In [8]:
ort_session.get_inputs()[4].shape

['N', 11, 'n_sv']

In [4]:
ort_session.get_outputs()[0].shape

['N', 1]

In [2]:
ort_session = onnxruntime.InferenceSession('particle-net.onnx')

2024-05-22 17:20:28.488666628 [E:onnxruntime:Default, env.cc:254 ThreadMain] pthread_setaffinity_np failed for thread: 15848, index: 6, mask: {14, 15, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2024-05-22 17:20:28.488663472 [E:onnxruntime:Default, env.cc:254 ThreadMain] pthread_setaffinity_np failed for thread: 15846, index: 4, mask: {10, 11, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2024-05-22 17:20:28.489013500 [E:onnxruntime:Default, env.cc:254 ThreadMain] pthread_setaffinity_np failed for thread: 15851, index: 9, mask: {20, 21, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2024-05-22 17:20:28.488894260 [E:onnxruntime:Default, env.cc:254 ThreadMain] pthread_setaffinity_np failed for thread: 15850, index: 8, mask: {18, 19, }, error code: 22 error msg: Invalid argument. S

In [11]:
d = torch.load("Datasets/stop_M600_588_ct200_2018.pt")

In [12]:
d[0]

Data(x=[41, 11], true_edges=[2, 2])

In [19]:
nt = d[0].x.unsqueeze(0)

In [23]:
nt.squeeze(0).shape

torch.Size([41, 11])